In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import disk, opening, closing
import glob
import time
import h5py

In [5]:
# Read input image and resize them
folder_in = 'imgs_samples/'
folder_out = 'imgs_samples_resized/'
images = sorted(glob.glob(folder_in + '*.jpg'))
n = 0
for img_path in images:
    cv2.imwrite(folder_out + 'resized' + str(n) + '.jpg', cv2.resize(cv2.imread(img_path), (0,0), fx=0.5, fy=0.5));
    n += 1

In [9]:
ILASTIK_PATH = '/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh'
MODEL='bees.ilp'
IMG = ''
OUTPUT_FOLDER = './imgs_samples_mask/{nickname}_mask.h5'
resized_imgs_path = sorted(glob.glob(folder_out + '*.jpg'))
for img_path in resized_imgs_path:
    IMG += img_path + ' '

!$ILASTIK_PATH  --headless \
                --project=$MODEL \
                --output_filename_format=$OUTPUT_FOLDER \
                $IMG

INFO ilastik.app: Using tiktorch executable: ['/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
Starting ilastik from "/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
WARNING 2024-04-06 18:16:33,708 opConservationTracking 66832 4517737984 Could not find any ILP solver
WARNING 2024-04-06 18:16:33,744 opStructuredTracking 66832 4517737984 Could not find any ILP solver
WARNING 2024-04-06 18:16:33,746 structuredTrackingWorkflow 66832 4517737984 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!
INFO ilastik.shell.projectManager: Opening Project: bees.ilp
INFO ilastik.workflows.pixelClassification.pixelClassificationWorkf

In [30]:
filenames = sorted(glob.glob('imgs_samples_mask/*.h5'))
n = 0
masks = []
for filename in filenames:
    with h5py.File(filename, "r") as f:
        a_group_key = list(f.keys())[0]
        data = list(f[a_group_key])
        ds_arr = f[a_group_key][:,:,0]  # returns as a numpy array
        cv2.imwrite('imgs_samples_mask/resized' + str(n) + '_mask.png', ds_arr)
        masks.append(cv2.imread('imgs_samples_mask/resized' + str(n) + '_mask.png', cv2.IMREAD_GRAYSCALE))
        n += 1

In [31]:
# morphological operations
final_masks = []
for mask in masks:
    mask_open = opening(mask, disk(9))
    mask_closed = closing(mask_open, disk(9))
    final_masks.append(mask_closed)


[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [

In [32]:
# now we can find the contours
imgs_contour = []
n = 0
resized_imgs = []
for img_path in resized_imgs_path:
    resized_imgs.append(cv2.imread(img_path))

for mask, input_image in zip(final_masks, resized_imgs):
    img_contour = (mask).astype(np.uint8) #convert to uint8 because findContours only accepts uint8
    contours, _ = cv2.findContours(img_contour, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #find contours
    contour = max(contours, key=cv2.contourArea) #get the biggest contour
    img_contour = cv2.cvtColor(img_contour, cv2.COLOR_GRAY2BGR) #convert to BGR because drawContours only accepts BGR
    cv2.drawContours(input_image, contour, -1, (255, 255, 255), 7) #draw the biggest contour

    # find the centroid of the contour
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    # plot it on the image
    cv2.circle(input_image, (cX, cY), 10, (255, 0, 0), -1)
    cv2.putText(input_image, "centroid", (cX - 30, cY - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.imwrite('imgs_samples_contour/contour' + str(n) + '.jpg', cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    n += 1